 # Imports

In [1]:
!pip install redis

  Obtaining dependency information for redis from https://files.pythonhosted.org/packages/0b/34/a01250ac1fc9bf9161e07956d2d580413106ce02d5591470130a25c599e3/redis-5.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for async-timeout>=4.0.2 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 5.1 MB/s eta 0:00:00a 0:00:01
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)


In [3]:
import numpy as np
import pandas as pd
import redis

# Configrue Redis Cloud

In [4]:
# redis-12830.c302.asia-northeast1-1.gce.cloud.redislabs.com:12830
# 0k137l1gcAW3TQMjUeQ06oc7vQTMKsAq

host = 'redis-12830.c302.asia-northeast1-1.gce.cloud.redislabs.com'
port_no = 12830
password = '0k137l1gcAW3TQMjUeQ06oc7vQTMKsAq'

r = redis.StrictRedis(host=host, port=port_no, password=password)

In [6]:
r.ping() # if true then connection successful

True

In [7]:
# r.keys()

[]

# Save Data to Redis from Numpy Zipped File

In [8]:
# load numpy zip and convert back to df
npz_file = np.load('df_students_teachers.npz', allow_pickle=True) # pickle helps in deserializing python objects and complex structures
print(type(npz_file.files))
npz_file.files

<class 'list'>


['arr_0', 'arr_1']

In [9]:
loaded_df = pd.DataFrame(npz_file['arr_0'], columns=npz_file['arr_1'])
loaded_df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.19883595, 1.4748771, -0.09800474, -0.880633..."
1,Barack Obama,Teacher,"[-0.22199515, 1.5761706, -0.5030543, 0.1360307..."
2,Chris Evans,Student,"[0.35688442, -0.41513723, 0.039245885, -0.1545..."
3,Morgan Freeman,Teacher,"[0.7628859, 0.89053726, 0.46874213, 0.2592453,..."
4,Scarlett Johansson,Student,"[-0.1354247, -0.28220648, -0.42639893, -0.2821..."


In [10]:
loaded_df['name_role'] = loaded_df['Name'] + '@' + loaded_df['Role']
loaded_df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.19883595, 1.4748771, -0.09800474, -0.880633...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[-0.22199515, 1.5761706, -0.5030543, 0.1360307...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.35688442, -0.41513723, 0.039245885, -0.1545...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.7628859, 0.89053726, 0.46874213, 0.2592453,...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[-0.1354247, -0.28220648, -0.42639893, -0.2821...",Scarlett Johansson@Student


In [13]:
records = loaded_df[['name_role', 'Facial_Features']].to_dict(orient='records') # records converts dict to list view
records

[{'name_role': 'Angelina Jolie@Student',
  'Facial_Features': array([ 1.98835954e-01,  1.47487712e+00, -9.80047435e-02, -8.80633175e-01,
         -4.95720476e-01,  6.36558652e-01,  7.20502973e-01,  4.32292402e-01,
         -1.01362300e+00, -2.19202489e-01, -1.25162446e+00,  6.23574853e-03,
         -1.08989584e+00, -9.10528958e-01,  9.79953408e-01, -8.08836937e-01,
         -3.21867943e-01, -3.24543655e-01, -1.63320929e-01, -2.94713378e-01,
          6.83515608e-01,  3.98809254e-01, -3.36343616e-01, -3.36887270e-01,
         -1.28304154e-01,  1.73715562e-01,  5.91930330e-01, -6.08671486e-01,
          1.05130899e+00,  7.50654221e-01,  5.73521852e-02, -4.44655001e-01,
          7.35095620e-01, -1.39628625e+00,  5.44071257e-01,  1.25621784e+00,
          1.07384071e-01, -3.43601614e-01, -1.05957888e-01,  1.95728517e+00,
          6.32352233e-01,  5.08020937e-01,  5.25747418e-01, -2.18284614e-02,
         -8.33760381e-01, -5.24998680e-02, -2.05504581e-01,  8.91154766e-01,
         -4.6965

In [14]:
len(records)

5

In [19]:
# saving records to redis as hashes
# in order to store facial features it has to be converted to bytes from array

for record in records:
    name_role = record['name_role']
    print(name_role)
    vector = record['Facial_Features']

    # convert array to bytes
    vector_bytes = vector.tobytes()
    # print(vector_bytes)

    # saving in redis cloud as hash
    r.hset(name='academy:register', key=name_role, value=vector_bytes)

Angelina Jolie@Student
Barack Obama@Teacher
Chris Evans@Student
Morgan Freeman@Teacher
Scarlett Johansson@Student


# Registration Form Prototype

In [20]:
import cv2
from insightface.app import FaceAnalysis

In [21]:
# buffalo_l model

model_l = FaceAnalysis(name='buffalo_l',
                        root='/Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l', 
                        providers=['CUDAExecutionProvider'])

model_l.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5) # input size

/Users/boss/miniconda3/envs/tensorflow/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/boss/Desktop/Notes/4_attendance_app/insightface_model/models/buffalo_l/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.

## Collect Info

### Name and Role

In [23]:
person_name = input('Enter you resgistrant name: ')

while True:
    role = int(input('Enter Choice: \n 1. Student \n 2.Teacher'))

    if role not in (1, 2):
        print('Try Again...')

    else:
        if role == 1:
            role = 'Student'
        else:
            role = 'Teacher'
        break

key = person_name + '@' + role
print('Name Entered: ', person_name)
print('Role Chosen: ', role)
print('Key Generated: ', key)

Name Entered:  Rahul
Role Chosen:  Teacher
Key Generated:  Rahul@Teacher


### Image Samples

In [26]:
cap = cv2.VideoCapture(0)
embedding_list = []
samples = 0

while True:
    ret, frame = cap.read()
    if ret == False:
        print('Camera Initialization Failed')
        break

    # insightface model inference   
    results = model_l.get(frame, max_num=1) # information for only one face
    for res in results:
        samples += 1
       
        x1, y1, x2, y2 = res['bbox'].astype(int)
        embedding = res['embedding']
        embedding_list.append(embedding)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0))
        if samples % 10 == 0:
         print(samples) # just for reference

    if samples >= 100:
        break
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()


/Users/boss/miniconda3/envs/tensorflow/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


10
20
30
40
50
60
70
80
90
100


In [27]:
facial_features = np.asarray(embedding_list).mean(axis=0)
facial_features.shape

(512,)

In [28]:
# convert numpy array to bytes
facial_features_bytes = facial_features.tobytes()

# save to cloud
r.hset(name='academy:register', key=key, value=facial_features_bytes) # academy:register is the name for hash

1

In [29]:
r.keys()

[b'academy:register']

**Next Phase is predictions in predictions notebook and main functions in face_recognition.py.**